In [ ]:
# Neurogenia
import importlib as imp
import neurogenia as genos
import time

from neurogenia import neuros

neuros = imp.reload(neuros)
genos = imp.reload(genos)

net = neuros.sequence([
    neuros.Layer(2),
    neuros.Activation('sigmoid'),
    neuros.Dense(5),
    neuros.Activation('sigmoid'),
    neuros.Dense(3)
])

print(net.predict([1, 1]))
print(list(net.predict([-1, -1])))
t1 = time.time()
f1 = net.target_fitness([[1, 1], [-1, -1]], [[-0.5, 0.9, 0.4], [-0.5, -1, 1]])
print(f1)
print("[...]")
n = genos.evolve_target(net, [[1, 1], [-1, -1]], [[-0.5, 0.9, 0.4], [-0.5, -1, 1]], population=100, mutation=0.3, elitism=0.08, survival=0.5, generations=200)
print("=====")
print(list(n.predict([1, 1])))
print(list(n.predict([-1, -1])))
f2 = n.target_fitness([[1, 1], [-1, -1]], [[-0.5, 0.9, 0.4], [-0.5, -1, 1]])
t2 = time.time()
print(f2)
print("Evolved {:.5f}% in {:.2f} seconds".format(((f2 - f1) * 100), t2 - t1))
# print(n is net)

[ 0.05451766  0.37643612 -0.36578384]
[0.2650943428995284, 0.3923107408915979, -0.32508594773473576]
-0.9079641971249471
[...]
Generation #     1 | Best: 62.0887% | Worst: -46.8051% | ETA: 10.9456s
Generation #     2 | Best: 68.6824% | Worst: -19.0874% | ETA: 9.8016s
Generation #     3 | Best: 66.5832% | Worst: 1.6708% | ETA: 9.5222s
Generation #     4 | Best: 69.4732% | Worst: -44.3179% | ETA: 9.3595s
Generation #     5 | Best: 71.2284% | Worst: 22.8860% | ETA: 9.2435s
Generation #     6 | Best: 69.5899% | Worst: 22.8209% | ETA: 9.4419s
Generation #     7 | Best: 72.1354% | Worst: -6.4255% | ETA: 9.3748s
Generation #     8 | Best: 75.9215% | Worst: -5.1275% | ETA: 9.2885s
Generation #     9 | Best: 73.2934% | Worst: 27.3168% | ETA: 9.1685s
Generation #    10 | Best: 73.0562% | Worst: 6.3113% | ETA: 9.1775s
Generation #    11 | Best: 73.3981% | Worst: 7.6713% | ETA: 9.2615s
Generation #    12 | Best: 78.2416% | Worst: -21.1480% | ETA: 9.1655s
Generation #    13 | Best: 77.9370% | Worst

Generation #   117 | Best: 99.5174% | Worst: 70.1255% | ETA: 3.6274s
Generation #   118 | Best: 99.6184% | Worst: 67.5962% | ETA: 3.5825s
Generation #   119 | Best: 99.3519% | Worst: 56.4545% | ETA: 3.5363s
Generation #   120 | Best: 99.4169% | Worst: 66.3692% | ETA: 3.4902s
Generation #   121 | Best: 99.4918% | Worst: 63.5405% | ETA: 3.4455s
Generation #   122 | Best: 99.2655% | Worst: 87.8817% | ETA: 3.4021s
Generation #   123 | Best: 99.0732% | Worst: 76.3714% | ETA: 3.3600s
Generation #   124 | Best: 99.7461% | Worst: 69.6769% | ETA: 3.3166s
Generation #   125 | Best: 99.4593% | Worst: 30.8342% | ETA: 3.2708s
Generation #   126 | Best: 99.3216% | Worst: 50.0866% | ETA: 3.2245s
Generation #   127 | Best: 99.0605% | Worst: 46.5819% | ETA: 3.1783s
Generation #   128 | Best: 99.5669% | Worst: 63.6817% | ETA: 3.1327s
Generation #   129 | Best: 99.5893% | Worst: 53.5386% | ETA: 3.0890s
Generation #   130 | Best: 99.6256% | Worst: 72.0538% | ETA: 3.0441s
Generation #   131 | Best: 99.5712

In [ ]:
# Per-character chatbot network
import importlib as imp
import numpy as np
import math
import time
import neurogenia as genos

from neurogenia import neuros

neuros = imp.reload(neuros)
genos = imp.reload(genos)


CHARVERSE = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789,.!?:;()"
MAX_RESPONSE_LEN = 80
MAX_INPUT_LEN = 75


net = neuros.sequence([
    neuros.Layer(MAX_INPUT_LEN),
    neuros.Activation('sigmoid'),
    neuros.Dense(int((MAX_INPUT_LEN + MAX_RESPONSE_LEN) / 2)),
    neuros.Activation('sigmoid'),
    neuros.Dense(MAX_RESPONSE_LEN),
    neuros.Activation('sigmoid')
])

def encode(text, max_len):
    res = []
    
    for i in range(max_len):
        # if i >= len(text):
        #     res.extend([0] * len(CHARVERSE))
        # else:
        #     c = [0] * len(CHARVERSE)
        #     if text[i] not in CHARVERSE:
        #         c[0] = 1
        #     else:
        #         c[CHARVERSE.index(text[i])] = 1
        #     res.extend(c)
            
        if i >= len(text):
            res.append(-1)
            
        else:
            if text[i] not in CHARVERSE:
                res.append(0)
            
            else:
                res.append(CHARVERSE.index(text[i]) / len(CHARVERSE))
    
    return res

def decode(text):
    res = ""
    
    for c in text:
        # m = max(c)
        # if m < 0.05:
        #     break
        # res += CHARVERSE[c.index(m)]
        
        if c < -0.2:
            break
        
        res += CHARVERSE[math.ceil(c * len(CHARVERSE))]
        
    return res

convs = [t.splitlines() for t in open('test_conversations.txt').read().split('\n\n')]

def train(conversations):
    inp = []
    out = []

    for conv in conversations:
        for i, sent in enumerate(conv[:-1]):
            inp.append(encode(sent[:MAX_INPUT_LEN], MAX_INPUT_LEN))
            out.append(encode(conv[i + 1][:MAX_RESPONSE_LEN], MAX_RESPONSE_LEN))
            print("'{}' -> '{}'".format(sent[:MAX_INPUT_LEN], conv[i + 1][:MAX_RESPONSE_LEN]))
            
    print("Training...")
    n = genos.evolve_target(net, inp, out, generations=40, population=50, elitism=0.08, survival=0.4, mutation=0.7)
    print("Trained!")

    return n
        
train(convs).predict(encode("Hey, sibling!", MAX_INPUT_LEN))